# Spark Wrangle Exercises

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

This exercises uses the `case.csv`, `dept.csv` and `source.csv` files from the san antonio 311 call dataset.

## 1.

Read the `case`, `department`, and `source` data into their own spark dataframes.

In [2]:
source = spark.read.csv("source.csv", sep=",", header=True, inferSchema=True)

In [3]:
source.show(5)

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
|   103582|     Carmen Cura|
|   106463| Richard Sanchez|
|   119403|  Betty De Hoyos|
|   119555|  Socorro Quiara|
+---------+----------------+
only showing top 5 rows



In [4]:
dept = spark.read.csv("dept.csv", sep=",", header=True, inferSchema=True)

In [5]:
dept.show(5)

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|                YES|
+--------------------+--------------------+----------------------+-------------------+
only showing top 5 rows



In [6]:
case = spark.read.csv("case.csv", sep=",", header=True, inferSchema=True)

In [7]:
case.show(2, vertical=True, truncate=False)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 1/1/18 0:42                          
 case_closed_date     | 1/1/18 12:29                         
 SLA_due_date         | 9/26/20 0:42                         
 case_late            | NO                                   
 num_days_late        | -998.5087616000001                   
 case_closed          | YES                                  
 dept_division        | Field Operations                     
 service_request_type | Stray Animal                         
 SLA_days             | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  EL PASO ST, San Antonio, 78207 
 council_district     | 5                                    
-RECORD 1----------------------------------------------------
 case_id

## 2.

Let's see how writing to the local disk works in spark:
- Write the code necessary to store the source data in both csv and json format, store these as `sources_csv` and `sources_json`

In [8]:
source.write.json("sources_json", mode="overwrite")

In [9]:
source.write.csv('sources_csv', mode='overwrite', header=True)

- Inspect your folder structure. What do you notice?
    - One file has no contents and is just named _SUCCESS
    - The other file has the contents of the dataframe

## 3.

Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

### Case

In [10]:
case.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



- Need to convert dates to datetime format
- Yes and no columns need to be converted to booleans (`case_closed`, `case_late`)
- `council_district` needs to be converted to string

In [11]:
# rename column to make naming consistent and more readable
case = case.withColumnRenamed("SLA_due_date", "case_due_date")

In [12]:
# Change date data types
print("--- Before handling dates")
case.select("case_opened_date", "case_closed_date", "case_due_date").show(5)

fmt = "M/d/yy H:mm"
case = (
    case.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_closed_date", to_timestamp("case_closed_date", fmt))
    .withColumn("case_due_date", to_timestamp("case_due_date", fmt))
)

print("--- After")
case.select("case_opened_date", "case_closed_date", "case_due_date").show(5)

--- Before handling dates
+----------------+----------------+-------------+
|case_opened_date|case_closed_date|case_due_date|
+----------------+----------------+-------------+
|     1/1/18 0:42|    1/1/18 12:29| 9/26/20 0:42|
|     1/1/18 0:46|     1/3/18 8:11|  1/5/18 8:30|
|     1/1/18 0:48|     1/2/18 7:57|  1/5/18 8:30|
|     1/1/18 1:29|     1/2/18 8:13| 1/17/18 8:30|
|     1/1/18 1:34|    1/1/18 13:29|  1/1/18 4:34|
+----------------+----------------+-------------+
only showing top 5 rows

--- After
+-------------------+-------------------+-------------------+
|   case_opened_date|   case_closed_date|      case_due_date|
+-------------------+-------------------+-------------------+
|2018-01-01 00:42:00|2018-01-01 12:29:00|2020-09-26 00:42:00|
|2018-01-01 00:46:00|2018-01-03 08:11:00|2018-01-05 08:30:00|
|2018-01-01 00:48:00|2018-01-02 07:57:00|2018-01-05 08:30:00|
|2018-01-01 01:29:00|2018-01-02 08:13:00|2018-01-17 08:30:00|
|2018-01-01 01:34:00|2018-01-01 13:29:00|2018-01-01 04:

In [13]:
# check yes and no columns to ensure there are only two categories
case.groupBy('case_late').count().show()

+---------+------+
|case_late| count|
+---------+------+
|      YES| 94503|
|       NO|747201|
+---------+------+



In [14]:
case.groupBy('case_closed').count().show()

+-----------+------+
|case_closed| count|
+-----------+------+
|        YES|823594|
|         NO| 18110|
+-----------+------+



In [15]:
case.groupBy('case_status').count().show()

+-----------+------+
|case_status| count|
+-----------+------+
|       Open| 18110|
|     Closed|823594|
+-----------+------+



In [16]:
# drop this column since it is same info as case_closed
case = case.drop('case_status')

In [17]:
# convert yes and no columns to booleans
case = case.withColumn("case_closed", expr('case_closed == "YES"')).withColumn(
    "case_late", expr('case_late == "YES"')
)

case.select("case_closed", "case_late").show(5)

+-----------+---------+
|case_closed|case_late|
+-----------+---------+
|       true|    false|
|       true|    false|
|       true|    false|
|       true|    false|
|       true|     true|
+-----------+---------+
only showing top 5 rows



In [18]:
# take a look at different values for council_district
case.groupBy('council_district').count().show()

+----------------+------+
|council_district| count|
+----------------+------+
|               1|119309|
|               6| 74095|
|               3|102706|
|               5|114609|
|               9| 40916|
|               4| 93778|
|               8| 42345|
|               7| 72445|
|              10| 62926|
|               2|114745|
|               0|  3830|
+----------------+------+



In [19]:
# convert council district to a string
case = case.withColumn("council_district", col("council_district").cast("string"))

In [20]:
case.show(2, vertical=True, truncate=False)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 2018-01-01 00:42:00                  
 case_closed_date     | 2018-01-01 12:29:00                  
 case_due_date        | 2020-09-26 00:42:00                  
 case_late            | false                                
 num_days_late        | -998.5087616000001                   
 case_closed          | true                                 
 dept_division        | Field Operations                     
 service_request_type | Stray Animal                         
 SLA_days             | 999.0                                
 source_id            | svcCRMLS                             
 request_address      | 2315  EL PASO ST, San Antonio, 78207 
 council_district     | 5                                    
-RECORD 1----------------------------------------------------
 case_id              | 1014127333                           
 case_op

In [21]:
case.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- case_due_date: timestamp (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = true)



### Source

In [22]:
source.show(5)

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
|   103582|     Carmen Cura|
|   106463| Richard Sanchez|
|   119403|  Betty De Hoyos|
|   119555|  Socorro Quiara|
+---------+----------------+
only showing top 5 rows



In [23]:
source.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)



No data type modifications needed

### Department

In [24]:
dept.show(5)

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|                YES|
+--------------------+--------------------+----------------------+-------------------+
only showing top 5 rows



In [25]:
dept.printSchema()

root
 |-- dept_division: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)



Convert last column to boolean

In [26]:
dept.groupBy('dept_subject_to_SLA').count().show()

+-------------------+-----+
|dept_subject_to_SLA|count|
+-------------------+-----+
|                YES|   31|
|                 NO|    8|
+-------------------+-----+



In [27]:
dept = dept.withColumn("dept_subject_to_SLA", expr('dept_subject_to_SLA == "YES"'))

dept.select("dept_subject_to_SLA").show(5)

+-------------------+
|dept_subject_to_SLA|
+-------------------+
|               true|
|               true|
|               true|
|               true|
|               true|
+-------------------+
only showing top 5 rows



In [28]:
dept.groupBy('dept_subject_to_SLA').count().show()

+-------------------+-----+
|dept_subject_to_SLA|count|
+-------------------+-----+
|               true|   31|
|              false|    8|
+-------------------+-----+



---

## 1.

- How old is the latest (in terms of days past SLA) currently open issue? 

In [29]:
case.where(case.case_closed == False).sort(case.num_days_late.desc()).show(1, vertical=True, truncate=False)

-RECORD 0------------------------------------------------------
 case_id              | 1013225646                             
 case_opened_date     | 2017-01-01 13:48:00                    
 case_closed_date     | null                                   
 case_due_date        | 2017-01-17 08:30:00                    
 case_late            | true                                   
 num_days_late        | 348.6458333                            
 case_closed          | false                                  
 dept_division        | Code Enforcement                       
 service_request_type | No Address Posted                      
 SLA_days             | 15.77859954                            
 source_id            | svcCRMSS                               
 request_address      | 7299  SHADOW RIDGE, San Antonio, 78250 
 council_district     | 6                                      
only showing top 1 row



In [30]:
case.where(case.case_closed == False).select(max(col('num_days_late'))).show()

+------------------+
|max(num_days_late)|
+------------------+
|       348.6458333|
+------------------+



- How long has the oldest (in terms of days since opened) currently opened issue been open?

In [31]:
case = (
    case.withColumn(
        "case_age", datediff(current_timestamp(), "case_opened_date")
    )
    .withColumn(
        "days_to_closed", datediff("case_closed_date", "case_opened_date")
    )
    .withColumn(
        "case_lifetime",
        when(expr("! case_closed"), col("case_age")).otherwise(
            col("days_to_closed")
        ),
    )
)

case.select(
    "case_closed",
    "case_opened_date",
    "case_closed_date",
    "case_age",
    "days_to_closed",
    "case_lifetime",
).where(expr("case_closed")).show(5)

case.select(
    "case_closed",
    "case_opened_date",
    "case_closed_date",
    "case_age",
    "days_to_closed",
    "case_lifetime",
).where(expr("! case_closed")).show(5)

+-----------+-------------------+-------------------+--------+--------------+-------------+
|case_closed|   case_opened_date|   case_closed_date|case_age|days_to_closed|case_lifetime|
+-----------+-------------------+-------------------+--------+--------------+-------------+
|       true|2018-01-01 00:42:00|2018-01-01 12:29:00|    1408|             0|            0|
|       true|2018-01-01 00:46:00|2018-01-03 08:11:00|    1408|             2|            2|
|       true|2018-01-01 00:48:00|2018-01-02 07:57:00|    1408|             1|            1|
|       true|2018-01-01 01:29:00|2018-01-02 08:13:00|    1408|             1|            1|
|       true|2018-01-01 01:34:00|2018-01-01 13:29:00|    1408|             0|            0|
+-----------+-------------------+-------------------+--------+--------------+-------------+
only showing top 5 rows

+-----------+-------------------+----------------+--------+--------------+-------------+
|case_closed|   case_opened_date|case_closed_date|case_age

In [32]:
case.where(case.case_closed == False).sort(case.case_lifetime.desc()).show(1, vertical=True, truncate=False)

-RECORD 0------------------------------------------------------
 case_id              | 1013225646                             
 case_opened_date     | 2017-01-01 13:48:00                    
 case_closed_date     | null                                   
 case_due_date        | 2017-01-17 08:30:00                    
 case_late            | true                                   
 num_days_late        | 348.6458333                            
 case_closed          | false                                  
 dept_division        | Code Enforcement                       
 service_request_type | No Address Posted                      
 SLA_days             | 15.77859954                            
 source_id            | svcCRMSS                               
 request_address      | 7299  SHADOW RIDGE, San Antonio, 78250 
 council_district     | 6                                      
 case_age             | 1773                                   
 days_to_closed       | null            

In [33]:
# Another way to do it:
(
    case.select(datediff(current_timestamp(), 'case_due_date')
    .alias('days_past_due'))
    .where(case.case_closed == False)
    .sort(col('days_past_due').desc())
    .show(5)
)

+-------------+
|days_past_due|
+-------------+
|         1757|
|         1757|
|         1757|
|         1756|
|         1754|
+-------------+
only showing top 5 rows



## 2.

How many Stray Animal cases are there?

In [34]:
case.where(expr('service_request_type == "Stray Animal"')).count()

26760

In [35]:
case.where(col('service_request_type') == 'Stray Animal').count()

26760

In [36]:
case.where(col('case_closed') == True).where(col('service_request_type') == 'Stray Animal').count()

26745

In [37]:
case.where(col('case_closed') == False).where(col('service_request_type') == 'Stray Animal').count()

15

## 3.

How many service requests that are assigned to the Field Operations department (`dept_division`) are not classified as "Officer Standby" request type (`service_request_type`)?

In [38]:
(
    case.filter(case.dept_division == 'Field Operations')
    .filter(case.service_request_type != 'Officer Standby')
    .count()
)

113902

## 4.

Convert the `council_district` column to a string column.

Already done in prep above

## 5.

Extract the year from the `case_closed_date` column.

In [39]:
case.select('case_closed_date', year('case_closed_date').alias('case_closed_year')).show(5)

+-------------------+----------------+
|   case_closed_date|case_closed_year|
+-------------------+----------------+
|2018-01-01 12:29:00|            2018|
|2018-01-03 08:11:00|            2018|
|2018-01-02 07:57:00|            2018|
|2018-01-02 08:13:00|            2018|
|2018-01-01 13:29:00|            2018|
+-------------------+----------------+
only showing top 5 rows



## 6.

Convert `num_days_late` from days to hours in new columns `num_hours_late`.

In [40]:
(
    case.withColumn('num_hours_late', case.num_days_late * 24)
    .select('num_days_late', 'num_hours_late')
    .show()
)

+-------------------+-------------------+
|      num_days_late|     num_hours_late|
+-------------------+-------------------+
| -998.5087616000001|     -23964.2102784|
|-2.0126041669999997|-48.302500007999996|
|       -3.022337963|      -72.536111112|
|       -15.01148148|      -360.27555552|
|0.37216435200000003|  8.931944448000001|
|       -29.74398148| -713.8555555199999|
|       -14.70673611|      -352.96166664|
|       -14.70662037|      -352.95888888|
|       -14.70662037|      -352.95888888|
|       -14.70649306|      -352.95583344|
|       -14.70649306|      -352.95583344|
|       -14.70636574|      -352.95277776|
|          -14.70625|-352.95000000000005|
|       -14.70636574|      -352.95277776|
|       -14.70623843|-352.94972232000003|
|-14.705891199999998|-352.94138879999997|
|       -14.70600694|      -352.94416656|
|       -14.70576389|      -352.93833336|
|       -14.70576389|      -352.93833336|
|       -14.70564815|       -352.9355556|
+-------------------+-------------

## 7.

Join the case data with the source and department data.

In [41]:
df = case.join(dept, 'dept_division', 'left').join(source, 'source_id', 'left')

In [42]:
df.show(2, vertical=True, truncate=False)

-RECORD 0------------------------------------------------------
 source_id              | svcCRMLS                             
 dept_division          | Field Operations                     
 case_id                | 1014127332                           
 case_opened_date       | 2018-01-01 00:42:00                  
 case_closed_date       | 2018-01-01 12:29:00                  
 case_due_date          | 2020-09-26 00:42:00                  
 case_late              | false                                
 num_days_late          | -998.5087616000001                   
 case_closed            | true                                 
 service_request_type   | Stray Animal                         
 SLA_days               | 999.0                                
 request_address        | 2315  EL PASO ST, San Antonio, 78207 
 council_district       | 5                                    
 case_age               | 1408                                 
 days_to_closed         | 0             

## 8.

Are there any cases that do not have a request source?

In [43]:
(
    df.select(df.source_id.isNull().cast('int').alias('is_null'))
    .agg(sum('is_null'))
    .show()
)

+------------+
|sum(is_null)|
+------------+
|           0|
+------------+



In [44]:
df.filter(col('source_id').isNull()).show(vertical=True)

(0 rows)



## 9.

What are the top 10 service request types in terms of number of requests?

In [45]:
(
    df.groupby('service_request_type')
    .count()
    .sort(col('count').desc())
    .show(10, truncate=False)
)

+--------------------------------+-----+
|service_request_type            |count|
+--------------------------------+-----+
|No Pickup                       |89210|
|Overgrown Yard/Trash            |66403|
|Bandit Signs                    |32968|
|Damaged Cart                    |31163|
|Front Or Side Yard Parking      |28920|
|Stray Animal                    |27361|
|Aggressive Animal(Non-Critical) |25492|
|Cart Exchange Request           |22608|
|Junk Vehicle On Private Property|21649|
|Pot Hole Repair                 |20827|
+--------------------------------+-----+
only showing top 10 rows



## 10.

What are the top 10 service request types in terms of average days late?

In [46]:
(
    df.where('case_late')
    .groupBy('service_request_type')
    .agg(mean('num_days_late').alias('n_days_late'), count('*').alias('n_cases'))
    .sort(desc('n_days_late'))
    .show(10, truncate=False)
)

+--------------------------------------+------------------+-------+
|service_request_type                  |n_days_late       |n_cases|
+--------------------------------------+------------------+-------+
|Zoning: Recycle Yard                  |210.89201994318182|132    |
|Zoning: Junk Yards                    |200.20517608494276|262    |
|Structure/Housing Maintenance         |190.20707698509804|51     |
|Donation Container Enforcement        |171.09115313942618|122    |
|Storage of Used Mattress              |163.96812829714287|7      |
|Labeling for Used Mattress            |162.43032902285717|7      |
|Record Keeping of Used Mattresses     |153.99724039428568|7      |
|Signage Requied for Sale of Used Mattr|151.63868055333333|12     |
|Traffic Signal Graffiti               |137.64583330000002|4      |
|License Requied Used Mattress Sales   |128.79828704142858|7      |
+--------------------------------------+------------------+-------+
only showing top 10 rows



## 11.

Does number of days late depend on department?

In [47]:
(
    df.filter('case_late')
    .groupby('dept_name')
    .agg(mean('num_days_late').alias('days_late'), count('num_days_late').alias('n_cases_late'))
    .sort('days_late')
    .withColumn('days_late', round(col('days_late'), 1))
    .sort(col('days_late').desc())
    .show(truncate=False)
)

+-------------------------+---------+------------+
|dept_name                |days_late|n_cases_late|
+-------------------------+---------+------------+
|null                     |210.9    |132         |
|Customer Service         |88.2     |2035        |
|Development Services     |67.2     |840         |
|Code Enforcement Services|48.1     |25467       |
|Animal Care Services     |23.4     |23751       |
|Parks and Recreation     |22.4     |3810        |
|Trans & Cap Improvements |10.7     |5529        |
|Solid Waste Management   |7.1      |33729       |
|Metro Health             |6.5      |854         |
+-------------------------+---------+------------+



## 12.

How do number of days late depend on department and request type?

In [48]:
(
    df.filter("case_closed")
    .filter("case_late")
    .groupby("standardized_dept_name", "service_request_type")
    .agg(avg("num_days_late").alias("days_late"), count("*").alias("n_cases"))
    .withColumn("days_late", round(col("days_late"), 1))
    .where(col('days_late') > 0)
    .sort(desc("days_late"))
    .show(40, truncate=False)
)

+----------------------+--------------------------------------------+---------+-------+
|standardized_dept_name|service_request_type                        |days_late|n_cases|
+----------------------+--------------------------------------------+---------+-------+
|DSD/Code Enforcement  |Zoning: Recycle Yard                        |273.6    |75     |
|DSD/Code Enforcement  |Zoning: Junk Yards                          |251.9    |146    |
|DSD/Code Enforcement  |Donation Container Enforcement              |201.7    |82     |
|DSD/Code Enforcement  |Structure/Housing Maintenance               |182.4    |30     |
|DSD/Code Enforcement  |Graffiti: Private Property (Corridors)      |175.1    |3      |
|DSD/Code Enforcement  |Storage of Used Mattress                    |164.0    |7      |
|DSD/Code Enforcement  |Labeling for Used Mattress                  |162.4    |7      |
|DSD/Code Enforcement  |Record Keeping of Used Mattresses           |154.0    |7      |
|DSD/Code Enforcement  |Signage 

---

You might have noticed that the latest date in the dataset is fairly far off from the present day. To account for this, replace any occurances of the current time with the maximum date from the dataset.